# Utility functions

In [1]:
from datasets.molecules.utils import smiles2graph
from GRALE.data import BatchedDenseData
import torch
from GRALE.vizualization.plots import graph_to_rdkit
from GRALE.main import GRALE_model
import json
from rdkit import Chem
from huggingface_hub import hf_hub_download


def smiles_to_data(smiles_list, config, device='cuda'):
    '''
    Convert a smiles list to a BatchedDenseData object.
    A BatchedDenseData object contains:
        - h: node mask (batch_size, n_max_nodes)
        - nodes.labels: node labels (batch_size, n_max_nodes, n_node_labels)
        - edges.adjacency: edge adjacency (batch_size, n_max_nodes, n_max_nodes)
        - edges.labels: edge labels (batch_size, n_max_nodes, n_max_nodes, n_edge_labels)
        - edges.SP: shortest path matrix (batch_size, n_max_nodes, n_max_nodes)
    '''
    n_max_nodes = config['n_max_nodes']
    valid_atomic_nums = config['valid_atomic_nums']
    n_node_labels = len(valid_atomic_nums)
    valid_bond_types = config['valid_bond_types']
    n_edge_labels= len(valid_bond_types)
    graphs = [smiles2graph(smiles,n_max_nodes,valid_atomic_nums,valid_bond_types) for smiles in smiles_list]
    h = torch.stack([~torch.tensor(graph['node_mask'],dtype=torch.bool,device=device) for graph in graphs])
    node_labels =  torch.stack([torch.nn.functional.one_hot(torch.tensor(graph['node_labels'], dtype=torch.long, device=device), num_classes=n_node_labels).to(torch.float32) for graph in graphs])
    edge_labels = torch.stack([torch.nn.functional.one_hot(torch.tensor(graph['edge_labels'], dtype=torch.long, device=device), num_classes=n_edge_labels).to(torch.float32) for graph in graphs])
    adjacency = torch.stack([torch.tensor(graph['edge_labels']>0, device=device, dtype=torch.float32) for graph in graphs])
    SP_matrix = torch.stack([torch.tensor(graph['SP_matrix'], device=device, dtype=torch.float32) for graph in graphs])
    data = BatchedDenseData(h=h, nodes={'labels': node_labels}, edges={'adjacency': adjacency, 'labels': edge_labels, 'SP': SP_matrix})
    return data

def data_to_smiles(batched_data, config):
    smiles_list = []
    for data in batched_data:
        h = data.h
        active = h > 0.5
        node_labels = data.nodes.labels[active].cpu().detach().numpy().argmax(-1)
        node_edges = data.edges.labels[active][:,active].cpu().detach().numpy().argmax(-1)
        smiles, mol = graph_to_rdkit(node_labels, node_edges)
        smiles_list.append(smiles)
    return smiles_list

def load_model(device='cuda'):
    out = hf_hub_download(repo_id="PaulKrzakala/GRALE-128-32", filename="last.ckpt")
    model = GRALE_model.load_from_checkpoint(out).to(device)
    config = json.load(open('datasets/molecules/configs/32.json','r'))
    return model, config

def accuracy(smiles_true, smiles_pred):
    correct = 0
    for s_true, s_pred in zip(smiles_true, smiles_pred):
        s_true_canonical = Chem.CanonSmiles(s_true)
        s_pred_canonical = Chem.CanonSmiles(s_pred)
        if s_true_canonical == s_pred_canonical:
            correct += 1
    return correct / len(smiles_true)


/Users/gabrielmelo/Documents/GRALE/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Compute Reconstruction Accuracy

In [2]:
from rdkit.Chem import MolFromSmiles, MolToSmiles

device=torch.device("mps")
# Load Model
model, config = load_model(device=device)
# Get SMILES strings
smiles_list = ['CN1C=NC2=C1C(=O)N(C(=O)N2C)C', 'CCN']
# Convert to BatchedDenseData format
data = smiles_to_data(smiles_list, config, device=device)
# Encode and Decode
latent = model.encode(data)
output = model.decode(latent)
# Convert back to SMILES
smiles_out = data_to_smiles(output, config)
# Compute reconstruction accuracy
print(f"Reconstruction accuracy: {accuracy(smiles_list, smiles_out):.2%}")

NotImplementedError: The operator 'aten::_linalg_eigh.eigenvalues' is not currently implemented for the MPS device. If you want this op to be considered for addition please comment on https://github.com/pytorch/pytorch/issues/141287 and mention use-case, that resulted in missing op as well as commit hash a1cb3cc05d46d198467bebbb6e8fba50a325d4e7. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

# Plot Reconstruction 

In [ ]:
from rdkit.Chem import MolFromSmiles
import matplotlib.pyplot as plt
from rdkit.Chem import Draw

# Load Model
model, config = load_model()
# Get SMILES strings
smiles_list = ['CN1C=NC2=C1C(=O)N(C(=O)N2C)C', 'CCN']
# Convert to BatchedDenseData format
data = smiles_to_data(smiles_list, config)
# Encode and Decode
latent = model.encode(data)
output = model.decode(latent)
smiles_out = data_to_smiles(output, config)

for smi_in, smi_out in zip(smiles_list, smiles_out):
    mol_in = MolFromSmiles(smi_in)
    mol_out = MolFromSmiles(smi_out)
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes[0].imshow(Draw.MolToImage(mol_in))
    axes[0].set_title('Input Molecule')
    axes[0].axis('off')
    axes[1].imshow(Draw.MolToImage(mol_out))
    axes[1].set_title('Reconstructed Molecule')
    axes[1].axis('off')
    plt.show()